<a href="https://colab.research.google.com/github/DilkiIshara/Hate-Speech-Detection-System/blob/main/Hate_Speech_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!nvidia-smi

Sat Aug 24 06:58:11 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [4]:
!pip install tensorflow

In [5]:
!pip install sentence-transformers

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [6]:
import csv
import re
from collections import defaultdict as ddict

import nltk
import numpy as np
import pandas as pd
import torch
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sentence_transformers import SentenceTransformer
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, Dataset
from transformers import (AdamW, XLMRobertaForSequenceClassification,
                          XLMRobertaTokenizer, get_linear_schedule_with_warmup)
from sklearn.feature_extraction.text import TfidfVectorizer
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


Load The Data set

In [7]:
sinhalaDataPath = '/content/drive/MyDrive/HSDData/SinhalaDataSet.tsv'


try:
    sinhala_train_data = pd.read_csv(sinhalaDataPath, delimiter='\t')
except pd.errors.EmptyDataError:
    print("The file is empty or could not be loaded.")
except pd.errors.ParserError as e:
    print(f"Parser Error: {e}")

Display all the row data

In [8]:
sinhala_train_data

,post_id,text,tokens,rationales,label
0,995411387894542336,තේ නෙවෙයි තෝ බීලා ඉන්නෙ ගිනි වතුර,තේ නෙවෙයි තෝ බීලා ඉන්නෙ ගිනි වතුර,[],NOT
1,1128238706303864832,@USER තුනක් ඕනේ නෑ එකක් ගැහුවනම් ඇති ලංකාවට ආප...,@USER තුනක් ඕනේ නෑ එකක් ගැහුවනම් ඇති ලංකාවට ආප...,[x],NOT
2,1290554471714496514,"@USER , @USER @USER ටහුඩු ඒකි තනියම ජීවත් වෙ...","@USER , @USER @USER ටහුඩු ඒකි තනියම ජීවත් වෙන්...",[],NOT
3,909459929446182913,@USER මර්විනුත් මුද්දරයක් වෙයිද දන්නේ නැ,@USER මර්විනුත් මුද්දරයක් වෙයිද දන්නේ නැ,[],NOT
4,1305480579375198208,@USER ගොසිප් පීපල් නෙමේ හුත්තො. තෝ වගේ හොරෙක්ට...,@USER ගොසිප් පීපල් නෙමේ හුත්තො . තෝ වගේ හොරෙක්...,"[0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]",OFF
...,...,...,...,...,...
2495,732381659518111744,"පට්ට වේසි, දැන් එනව මෙතන රහත් වෙන්න URL","පට්ට වේසි , දැන් එනව මෙතන රහත් වෙන්න URL","[1, 1, 0, 0, 0, 0, 0, 0, 0]",OFF
2496,944199723564650496,"මේක හරි හුත්තක් උනානේ ""කොල්ලෙක් නෑ ,කෙල්ලෙක් ...","මේක හරි හුත්තක් උනානේ "" කොල්ලෙක් නෑ , කෙල්ලෙක්...","[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",OFF
2497,680949697666781184,ලොකු නංඟි ගෙ පරණ කොල්ලා ඒකිට වද දෙනවලු ඒ වුනාට...,ලොකු නංඟි ගෙ පරණ කොල්ලා ඒකිට වද දෙනවලු ඒ වුනාට...,[],NOT
2498,1184506690122633216,බය කරන්න එපා යකෝ සිත්තරා @USER · 16 Oct 2019...,බය කරන්න එපා යකෝ සිත්තරා @USER 16 Oct 2019 මේය...,[],NOT


Clean Up the data Set

In [9]:
def cleaning_text(df, text_field):
  df[text_field] = df[text_field].apply(lambda elem: re.sub(r"(@[A-Za-z0-9]+)|#|([A-Za-z0-9])|(\.+)|\t.+", "", elem))
  return df

train = cleaning_text(sinhala_train_data, "text")

Diaplay the clean Up data

In [10]:
train

,post_id,text,tokens,rationales,label
0,995411387894542336,තේ නෙවෙයි තෝ බීලා ඉන්නෙ ගිනි වතුර,තේ නෙවෙයි තෝ බීලා ඉන්නෙ ගිනි වතුර,[],NOT
1,1128238706303864832,තුනක් ඕනේ නෑ එකක් ගැහුවනම් ඇති ලංකාවට ආපු ගමන...,@USER තුනක් ඕනේ නෑ එකක් ගැහුවනම් ඇති ලංකාවට ආප...,[x],NOT
2,1290554471714496514,", ටහුඩු ඒකි තනියම ජීවත් වෙන්න වෙරදරන එක්ක...","@USER , @USER @USER ටහුඩු ඒකි තනියම ජීවත් වෙන්...",[],NOT
3,909459929446182913,මර්විනුත් මුද්දරයක් වෙයිද දන්නේ නැ,@USER මර්විනුත් මුද්දරයක් වෙයිද දන්නේ නැ,[],NOT
4,1305480579375198208,ගොසිප් පීපල් නෙමේ හුත්තො තෝ වගේ හොරෙක්ට මේකෙ ...,@USER ගොසිප් පීපල් නෙමේ හුත්තො . තෝ වගේ හොරෙක්...,"[0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]",OFF
...,...,...,...,...,...
2495,732381659518111744,"පට්ට වේසි, දැන් එනව මෙතන රහත් වෙන්න","පට්ට වේසි , දැන් එනව මෙතන රහත් වෙන්න URL","[1, 1, 0, 0, 0, 0, 0, 0, 0]",OFF
2496,944199723564650496,"මේක හරි හුත්තක් උනානේ ""කොල්ලෙක් නෑ ,කෙල්ලෙක් ...","මේක හරි හුත්තක් උනානේ "" කොල්ලෙක් නෑ , කෙල්ලෙක්...","[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",OFF
2497,680949697666781184,ලොකු නංඟි ගෙ පරණ කොල්ලා ඒකිට වද දෙනවලු ඒ වුනාට...,ලොකු නංඟි ගෙ පරණ කොල්ලා ඒකිට වද දෙනවලු ඒ වුනාට...,[],NOT
2498,1184506690122633216,බය කරන්න එපා යකෝ සිත්තරා · මේය ලොව ඇති භ...,බය කරන්න එපා යකෝ සිත්තරා @USER 16 Oct 2019 මේය...,[],NOT


Save the clean Up data in train.tsv

In [11]:
output_filename = '/content/drive/MyDrive/HSDData/train.tsv'
train.to_csv(output_filename, sep='\t', index=False, header=True, encoding='utf-8')

Removing Stopwords

In [12]:
def tfidf_feature(df, text_field):
  tfidf_vectorizer = TfidfVectorizer(max_features=1000)
  tfidf_features = tfidf_vectorizer.fit_transform(df[text_field])
  tfidf_df = pd.DataFrame(tfidf_features.toarray(), columns=tfidf_vectorizer.get_feature_names_out())
  df_with_tfidf = pd.concat([df, tfidf_df], axis=1)
  return(df_with_tfidf)

after_removeStopWord = tfidf_feature(train, 'text')
print(after_removeStopWord)

                  post_id                                               text  \
0      995411387894542336                  තේ නෙවෙයි තෝ බීලා ඉන්නෙ ගිනි වතුර   
1     1128238706303864832   තුනක් ඕනේ නෑ එකක් ගැහුවනම් ඇති ලංකාවට ආපු ගමන...   
2     1290554471714496514   ,     ටහුඩු ඒකි තනියම ජීවත් වෙන්න වෙරදරන එක්ක...   
3      909459929446182913                 මර්විනුත් මුද්දරයක් වෙයිද දන්නේ නැ   
4     1305480579375198208   ගොසිප් පීපල් නෙමේ හුත්තො තෝ වගේ හොරෙක්ට මේකෙ ...   
...                   ...                                                ...   
2495   732381659518111744               පට්ට වේසි, දැන් එනව මෙතන රහත් වෙන්න    
2496   944199723564650496  මේක හරි හුත්තක් උනානේ  "කොල්ලෙක් නෑ ,කෙල්ලෙක් ...   
2497   680949697666781184  ලොකු නංඟි ගෙ පරණ කොල්ලා ඒකිට වද දෙනවලු ඒ වුනාට...   
2498  1184506690122633216  බය කරන්න එපා යකෝ  සිත්තරා   ·    මේය ලොව ඇති භ...   
2499   803243627493269505   හොදට සෝදලා ගැබුරු තෙලේ බැදලා ගත්තම රසයි එහෙම ...   

                                       

saving this in the drive only for the purpose of crosschecking if all the necessary info exists

In [13]:
output_filename = '/content/drive/MyDrive/HSDData/afterRemoveStopWord.tsv'
after_removeStopWord.to_csv(output_filename, sep='\t', index=False, encoding='utf-8')

Split the Data into training and testing sets and save in separate files

In [14]:
trainDataSetFileName = '/content/drive/MyDrive/HSDData/train_DataSet.tsv'
testDataSetFileName = '/content/drive/MyDrive/HSDData/test_DataSet.tsv'

train_DataSet = train.sample(frac=0.8, random_state=73)
test_DataSet = train.drop(train_DataSet.index)


train_DataSet.to_csv(trainDataSetFileName,sep='\t', index=False)
test_DataSet.to_csv(testDataSetFileName, sep='\t',index=False)

In [15]:
label_categories = {
    'NOT':0,
    'OFF':1
}

def load_dataset():
  lines = []
  with open(trainDataSetFileName, "r", encoding='utf-8') as f:
    reader = csv.reader(f, delimiter='\t')
    for line in reader:
      lines.append(line)
      # print("Data" + line[0])
    texts=[]
    label =[]
    dic_data = ddict(list)
    for (i,line) in enumerate(lines):
      if i==0:
        continue

      text = line[1]
      labelValue = line[4]

      # print("Data" + text)
      # print("Label" + labelValue)

      texts.append(text)
      label.append(label_categories[labelValue])
    label_list = ["OFF", "NOT"]
    dic_data['texts']=texts
    dic_data['label']=label
    print(len(texts))
    print(len(label))
    return dic_data,label_list

In [16]:
train_DataList, label_List= load_dataset()

2000
2000


In [17]:
data = pd.DataFrame(
    {
        'Data' : train_DataList['texts'],
        'Label' : train_DataList['label']
    }
)

data


,Data,Label
0,පාර්ලිමේන්තු ක්‍රමයක් යටතේ මේ රට හදනවා කියන එක...,0
1,රජය සහ අාන්ඩුව කියන වචනවල තේරුම දන්නෙ නැති මන...,0
2,යුටියුබ් එකේ පලකරල තිබුනේ ආණ්ඩු පක්ෂයේ දෙදෙනෙ...,0
3,අඩො අර බ්‍රැන්ඩික්ස් කොරොනා පොකුරද කොලඹ මහ නගර...,0
4,"මේවා කැත වැඩ, නිමේස් ගැනත් හිතන්න පොඩ්ඩක්",0
...,...,...
1995,වාහන සොයා ඊයෙ දකුණු රටේ සංචාරය කරාදවසම කාලකන්න...,1
1996,ගනේමුල්ල කොහෙද ගියෙ බොල,0
1997,මොකද ඔච්චර කොටට අදින්නේඕකත් නැතුව හිටපංකො : ත...,1
1998,ඇයි බං මේ අපි අකමැත්තෙන් එක එක පාඩම් ඉගෙනගන්නෙ...,1


In [18]:
texts = data['Data'].tolist()
labels = data['Label'].tolist()

tokenizer = Tokenizer(num_words=10000)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

max_sequence_length = max(len(seq) for seq in sequences)
padded_sequences = pad_sequences(sequences, maxlen=max_sequence_length)

labels = np.array(labels)

model = Sequential()
model.add(Embedding(input_dim=10000, output_dim=128, input_length=max_sequence_length))
model.add(LSTM(units=256, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(units=1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Print the model architecture summary
model.summary()

model.fit(padded_sequences, labels, epochs=5, batch_size=64, validation_split=0.2)

model = Sequential()
model.add(Embedding(input_dim=10000, output_dim=128, input_length=max_sequence_length))
model.add(LSTM(units=128, dropout=0.2, recurrent_dropout=0.2, return_sequences=True))
model.add(Dense(units=1, activation='sigmoid'))
model.add(LSTM(units=64, dropout=0.2, recurrent_dropout=0.2))  # Additional LSTM layer
model.add(Dense(units=1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

model.fit(padded_sequences, labels, epochs=5, batch_size=64, validation_split=0.2)

model = Sequential()
model.add(Embedding(input_dim=10000, output_dim=128, input_length=max_sequence_length))
model.add(LSTM(units=256, dropout=0.2, recurrent_dropout=0.2, return_sequences=True))
model.add(LSTM(units=128, dropout=0.2, recurrent_dropout=0.2))  # Additional LSTM layer
model.add(Dense(units=1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

model.fit(padded_sequences, labels, epochs=5, batch_size=64, validation_split=0.2)

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_2 (Embedding)              │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_3 (LSTM)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_4 (LSTM)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

Epoch 1/5
25/25 ━━━━━━━━━━━━━━━━━━━━ 11s 325ms/step - accuracy: 0.5815 - loss: 0.6725 - val_accuracy: 0.5900 - val_loss: 0.6789
Epoch 2/5
25/25 ━━━━━━━━━━━━━━━━━━━━ 7s 211ms/step - accuracy: 0.7024 - loss: 0.6493 - val_accuracy: 0.7175 - val_loss: 0.5794
Epoch 3/5
25/25 ━━━━━━━━━━━━━━━━━━━━ 10s 210ms/step - accuracy: 0.9180 - loss: 0.2865 - val_accuracy: 0.7525 - val_loss: 0.6329
Epoch 4/5
25/25 ━━━━━━━━━━━━━━━━━━━━ 11s 221ms/step - accuracy: 0.9711 - loss: 0.0820 - val_accuracy: 0.7425 - val_loss: 0.7655
Epoch 5/5
25/25 ━━━━━━━━━━━━━━━━━━━━ 5s 212ms/step - accuracy: 0.9889 - loss: 0.0369 - val_accuracy: 0.7350 - val_loss: 0.9193


In [19]:
label_categories = {
    'NOT':0,
    'OFF':1
}

testDataSetFileName = '/content/drive/MyDrive/HSDData/test_DataSet.tsv'

def load_dataset_test():
  lines = []
  with open(testDataSetFileName, "r", encoding='utf-8') as f:
    reader = csv.reader(f, delimiter='\t')
    for line in reader:
      lines.append(line)
    texts=[]
    label =[]
    dic_data = ddict(list)
    for (i,line) in enumerate(lines):
      if i==0:
        continue

      textValue = line[1]
      labelValue = line[4]

      texts.append(textValue)
      label.append(label_categories[labelValue])
    label_list1 = ["OFF", "NOT"]
    dic_data['texts']=texts
    dic_data['label']=label
    return dic_data,label_list1
test_d, label_list1= load_dataset_test()

/////upsdatevvv

In [24]:
data_test = pd.DataFrame(
    {
        'Data' : test_d['texts'],
        'Label' : test_d['label']
    }
)
test_texts = data_test['Data'].tolist()
test_labels = data_test['Label'].tolist()
test_sequences = tokenizer.texts_to_sequences(test_texts)
padded_test_sequences = pad_sequences(test_sequences, maxlen=max_sequence_length)
test_labels = np.array(test_labels)

from sklearn.metrics import accuracy_score, f1_score
test_predictions = model.predict(padded_test_sequences)
test_predictions = (test_predictions > 0.5).astype(int)
test_accuracy = accuracy_score(test_labels, test_predictions)
test_f1 = f1_score(test_labels, test_predictions)

print(f'Testing Accuracy: {test_accuracy:.4f}')
print(f'Testing F1 Score: {test_f1:.4f}')

16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
Testing Accuracy: 92.0000
Testing Accuracy: 0.7020
Testing F1 Score: 0.6089


////////////////////////////////////////////////////////////////
/////////////////////////////////////////////////////////////////
//////////////////////////////////////////////////////////////////
/////////////////////////////////////////////////////////////////
//////////////////////////////////////////////////////////////////
//////////////////////////////////////////////////////////////////

In [21]:
from tensorflow.keras.layers import Conv1D, MaxPooling1D, GlobalMaxPooling1D

model = Sequential()

# Add an Embedding layer
model.add(Embedding(input_dim=10000, output_dim=128, input_length=max_sequence_length))

# Add a Conv1D layer
model.add(Conv1D(filters=64, kernel_size=5, activation='relu'))
model.add(MaxPooling1D(pool_size=2))

# Add an LSTM layer
model.add(LSTM(units=128, dropout=0.2, recurrent_dropout=0.2))
# Add Dense layers
model.add(Dense(units=64, activation='relu'))
model.add(Dense(units=1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()

model.fit(padded_sequences, labels, epochs=5, batch_size=64, validation_split=0.2)

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_3 (Embedding)              │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d (Conv1D)                      │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d (MaxPooling1D)         │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_5 (LSTM)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

Epoch 1/5
25/25 ━━━━━━━━━━━━━━━━━━━━ 6s 99ms/step - accuracy: 0.5457 - loss: 0.6828 - val_accuracy: 0.5850 - val_loss: 0.6764
Epoch 2/5
25/25 ━━━━━━━━━━━━━━━━━━━━ 4s 90ms/step - accuracy: 0.6000 - loss: 0.6561 - val_accuracy: 0.6950 - val_loss: 0.6147
Epoch 3/5
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 98ms/step - accuracy: 0.9017 - loss: 0.3610 - val_accuracy: 0.7300 - val_loss: 0.6473
Epoch 4/5
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 62ms/step - accuracy: 0.9810 - loss: 0.0591 - val_accuracy: 0.7200 - val_loss: 0.8562
Epoch 5/5
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 62ms/step - accuracy: 0.9944 - loss: 0.0182 - val_accuracy: 0.7075 - val_loss: 1.1410
